# Text2SQL : 성능 평가 파이프라인 준비하기

## SQL 생성 프롬프트

**SQL 프롬프트**

In [3]:
def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.
### DDL:
{ddl}

### Question:
{question}

### SQL:
{query}"""
    return prompt

## GPT-4 평가 프롬프트와 코드 준비

**평가를 위한 요청 jsonl 작성 함수**

In [4]:
import json
from pathlib import Path

In [5]:
def make_requests_for_gpt_evaluation(df, filename, dir='requests'):
    if not Path(dir).exists():
        Path(dir).mkdir(parents=True)
    prompts = []
    for idx, row in df.iterrows():
        prompts.append("""Based on below DDL and Question, evaluate gen_sql can resolve Question. If gen_sql and gt_sql do equal job, return "yes" else "no". Output Json Format: {"resolve_yn": ""}""" + f"""

DDL: {row['context']}
Question: {row['question']}
gt_sql: {row['answer']}
gen_sql: {row['gen_sql']}"""
)

    jobs = [{"model": "gpt-4-turbo", "response_format": {"type": "json_object"}, "messages": [{"role": "system", "content": prompt}]} for prompt in prompts]
    with open(Path(dir, filename), "w") as f:
        for job in jobs:
            json_string = json.dumps(job)
            f.write(json_string + "\n")

**비동기 요청 명령**

In [6]:
import os
import openai

```python
python api_request_parallel_processor.py \
    --requests_filepath {요청 파일 경로} \
    --save_filepath {생성할 결과 파일 경로} \
    --request_url https://api.openai.com/v1/chat/completions \
    --max_requests_per_minute 300 \
    --max_tokens_per_minute 100000 \
    --token_encoding_name cl100k_base \
    --max_attempts 5 \
    --logging_level 20
```
- --requests_filepath {요청 파일 경로}:
    - 요청 파일의 경로를 지정합니다. 이 파일은 미리 준비된 API 요청을 포함하고 있으며, 이 파일을 기반으로 OpenAI API로 병렬 요청을 보냅니다. 이 요청 파일은 앞서 설명한 make_requests_for_gpt_evaluation 함수에서 생성한 파일이 될 것입니다.
- --save_filepath {생성할 결과 파일 경로}:
    - API 응답 결과를 저장할 파일 경로를 지정합니다. 각 요청에 대한 응답이 JSON 형식으로 이 파일에 기록됩니다.
- --request_url https://api.openai.com/v1/chat/completions:
    - GPT API의 요청 URL입니다. 여기서는 GPT-4 API를 사용하고 있으며, chat/completions 엔드포인트는 채팅 기반 모델의 응답을 받기 위한 엔드포인트입니다.
- --max_requests_per_minute 300:
    - 분당 최대 요청 수를 제한합니다. API 서버의 요청 속도를 제어하여, 과도한 요청으로 인한 속도 제한(rate limiting)을 방지합니다. 이 예시에서는 분당 최대 300개의 요청을 허용하고 있습니다.
- --max_tokens_per_minute 100000:
    - 분당 최대 토큰 수를 제한합니다. OpenAI의 API 사용 시 응답의 토큰 수에 따라 비용이 발생하므로, 토큰 사용량을 제어하여 효율적인 요청을 보낼 수 있습니다. 여기서는 분당 최대 100,000 토큰을 허용하고 있습니다.
- --token_encoding_name cl100k_base:
    - 토큰 인코딩 방식을 지정합니다. OpenAI의 모델에서 사용하는 토큰화 알고리즘을 명시하며, 여기서는 cl100k_base라는 인코딩 방식을 사용하고 있습니다. 이 방식은 GPT-4와 호환되는 인코딩 방식입니다.
- --max_attempts 5:
    - 최대 재시도 횟수를 지정합니다. 네트워크 오류나 API 오류로 인해 요청이 실패할 경우, 최대 5회까지 재시도를 허용하여 요청의 성공 가능성을 높입니다.
- --logging_level 20:
    - 로그 레벨을 설정합니다. 20은 INFO 수준의 로그를 의미하며, 이 설정은 진행 상황 및 오류 메시지를 적절한 수준으로 출력하게 됩니다.
    - 주요 로그 레벨:
        - 10: DEBUG (상세한 로그)
        - 20: INFO (기본적인 정보 출력)
        - 30: WARNING (경고 메시지)
        - 40: ERROR (에러 메시지)
        - 50: CRITICAL (심각한 에러 메시지)

**결과 jsonl 파일을 csv로 변환하는 함수**

In [9]:
def change_jsonl_to_csv(input_file, output_file, prompt_column="prompt", response_column="response"):
    prompts = []
    responses = []
    with open(input_file, 'r') as json_file:
        for data in json_file:
            prompts.append(json.loads(data)[0]['messages'][0]['content'])
            responses.append(json.loads(data)[1]['choices'][0]['message']['content'])

    df = pd.DataFrame({prompt_column: prompts, response_column: responses})
    df.to_csv(output_file, index=False)
    return df

# 미세 조정 수행하기

## 기초 모델 평가하기

**기초 모델로 생성하기**

In [10]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

2024-10-22 05:44:38.005229: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-22 05:44:38.013392: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-22 05:44:38.022798: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-22 05:44:38.025581: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-22 05:44:38.032980: I tensorflow/core/platform/cpu_feature_guar

In [11]:
def make_inference_pipeline(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
    return pipe

In [ ]:
model_id = 'beomi/Yi-Ko-6B'
hf_pipe = make_inference_pipeline(model_id)

example = """당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
CREATE TABLE players (
    player_id INT PRIMARY KEY AUTO_INCREMENT,
    username VARCHAR(255) UNIQUE NOT NULL,
    email VARCHAR(255) UNIQUE NOT NULL,
    password_hash VARCHAR(255) NOT NULL,
    date_joined DATETIME NOT NULL,
    last_login DATETIME
);

### Question:
사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.

### SQL:
"""

hf_pipe(example, do_sample=False, return_full_text=False, max_length=1024, truncation=True)

In [12]:
model_id = 'beomi/Yi-Ko-6B'
hf_pipe = make_inference_pipeline(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

**기초 모델 성능 측정**

In [13]:
from datasets import load_dataset

In [14]:
# 데이터셋 불러오기
df = load_dataset("shangrilar/ko_text2sql", "origin")['test']
df = df.to_pandas()
for idx, row in df.iterrows():
    prompt = make_prompt(row['context'], row['question'])
    df.loc[idx, 'prompt'] = prompt

In [15]:
df.head()

,db_id,context,question,answer,prompt
0,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,각 보상 아이템별로 보상 경험치의 합을 구해줘,"SELECT reward_items, SUM(reward_experience) AS...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
1,1,CREATE TABLE players (\n player_id INT PRIMAR...,사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.,SELECT COUNT(*) FROM players WHERE username LI...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
2,1,CREATE TABLE quests (\n quest_id INT PRIMARY ...,퀘스트 진행 상황이 100%인 퀘스트의 이름과 보상 경험치는 얼마인가요?,"SELECT q.name, q.reward_experience FROM quests...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
3,1,CREATE TABLE characters (\n character_id INT ...,경험이 5000000 이상이거나 직업이 전사인 캐릭터들의 이름은 무엇인가,SELECT name FROM characters WHERE experience >...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
4,1,CREATE TABLE characters (\n character_id INT ...,레벨이 20 이상인 플레이어의 캐릭터 이름과 해당 캐릭터의 스킬 이름을 알아보세요.,"SELECT C.name, ST.skill_name FROM characters A...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...


In [16]:
# sql 생성
gen_sqls = hf_pipe(df['prompt'].tolist(), do_sample=False, return_full_text=False, max_length=1024, truncation=True)
gen_sqls = [x[0]['generated_text'] for x in gen_sqls]
df['gen_sql'] = gen_sqls

In [17]:
# 평가를 위한 requests.jsonl 생성
eval_filepath = "text2sql_evalutation.jsonl"
make_requests_for_gpt_evaluation(df, eval_filepath)

In [ ]:
# GPT-4 평가 수행
!python3 api_request_parallel_processor.py \
    --requests_filepath requests/{eval_filepath} \
    --save_filepath results/{eval_filepath} \
    --request_url https://api.openai.com/v1/chat/completions \
    --max_requests_per_minute 2500 \
    --max_tokens_per_minute 100000 \
    --token_encoding_name cl100k_base \
    --max_attempts 5 \
    --logging_level 20 

- 현재 무료 크레딧이 없으므로 API 사용 불가

## 미세 조정 수행

**학습 데이터 불러오기**

In [23]:
from datasets import load_dataset

In [ ]:
df_sql = load_dataset("shangrilar/ko_text2sql", "origin")["train"]
df_sql = df_sql.to_pandas()
df_sql = df_sql.dropna().sample(frac=1, random_state=42)
df_sql = df_sql.query("db_id != 1") # 데이터셋에서 평가에 사용하기로 한 db_id가 1인 데이터를 제거

for idx, row in df_sql.iterrows():
    df_sql.loc[idx, 'text'] = make_prompt(row['context'], row['question'], row['answer'])

!mkdir data
df_sql.to_csv('data/train.csv', index=False)

**미세 조정 명령어**

In [ ]:
base_model = 'beomo/Yi-Ko-6B'
finetuned_model = 'yi-ko-6b-text2sql'

In [ ]:
!autotrain llm \
--train \
--model {base_model} \
--project-name {finetuned-model} \
--data-path data/ \
--text-column test \
--lr 2e-4 \
--batch-size 8 \
--epochs 1 \
--block-size 1024 \
--warmup-ratio 0.1 \
--lora-r 16 \
--lora-alpha 32 \
--lora-dropout 0.05 \
--weight-decay 0.01 \
--gradient-accumulation 8 \
--mixed-precision fp16 \
--use-peft \
--quantization int4 \
--trainer sft

**LoRA 어댑터 결합 및 허깅페이스 허브 업로드**

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, PeftModel

In [ ]:
model_name = start_model
device_map = {"": 0}

# LoRA와 기초 모델 파라미터 합치기
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 허깅페이스 허브에 모델 및 토크나이저 저장
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

**미세 조정한 모델로 예시 데이터에 대한 SQL 생성**

In [ ]:
model_id = "shargrilar/yi-ko-6b-text2sql"
hf_pipe = make_inference_pipeline(model_id)

hf_pipe(example, do_sample=False,
        return_full_text=False, max_length=1024, truncation=True)

**미세 조정한 모델 성능 측정**

In [ ]:
# sql 생성 수행
gen_sqls = hf_pipe(df['prompt'].tolist(), do_sample=False,
                   return_full_text=False, max_length=1024, truncation=True)
gen_sqls = [x[0]['generated_text'] for x in gen_sqls]
df['gen_sql'] = gen_sqls

# 평가를 위한 requests.jsonl 생성
eval_filepath = "text2sql_evaluation_finetuned.jsonl"
make_requests_for_gpt_evaluation(df, eval_filepath)

In [ ]:
# GPT-4 평가 수행
!python3 api_request_parallel_processor.py \
    --requests_filepath requests/{eval_filepath} \
    --save_filepath results/{eval_filepath} \
    --request_url https://api.openai.com/v1/chat/completions \
    --max_requests_per_minute 2500 \
    --max_tokens_per_minute 100000 \
    --token_encoding_name cl100k_base \
    --max_attempts 5 \
    --logging_level 20 